In [2]:
import math
import random
import copy
import itertools as it
TAMANHOTABULEIRODEFAULT = 8

def geraTabuleiro(tamanho):
    if(tamanho < 2):
        return [-1]
    tab = []
    for i in range(tamanho):
        l = []
        for j in range(tamanho):
            l.append(0)
        tab.append(l)
    for i in range(tamanho):
        while(True):
            j = random.randint(0,tamanho-1)
            if(tab[i][j] == 0):
                tab[i][j] = 1
                break
    return tab

def locateQueens(estado):
    q = []
    for i in range(len(estado)):
        for j in range(len(estado)):
            if(estado[i][j]==1):
                q.append((i,j))
    return q

def calcAtaques(estado):
    at = 0
    #ataques em linhas
    atlinhas = 0
    for i in estado:
        if(sum(i)>1):
            atlinhas+=sum(i)-1
    at+=atlinhas
    #ataques em colunas
    atcolunas = 0
    for c in range(len(estado)):
        nl = 0
        for l in range(len(estado)):
            nl += estado[l][c]
        if(nl>1):
            atcolunas += nl - 1
    at+=atcolunas
    #ataques em diagonais
    pr = locateQueens(estado)
    d = 0
    for p,coords in enumerate(pr):
        i,j = coords
        #diagonal esquerda
        k,w = i+1,j-1
        while(w>=0 and k<len(estado)):
            if(estado[k][w]==1):
                d+=1
                break
            w-=1
            k+=1
        #diagonal direita
        l,c = i+1,j+1
        while(l<len(estado) and c<len(estado)):
            if(estado[l][c]==1):
                d+=1
                break
            l+=1
            c+=1
    at+=d
    return at

def mutacao(estado,passo):
    tam = len(estado)
    while(True):
        i,j = random.choice(locateQueens(estado))
        k = (j + passo)%tam
        if(estado[i][k]==0):
            estado[i][k],estado[i][j] = estado[i][j],estado[i][k]
            break
    return estado

def geraPopulacaoInicial(N,tam):
    #N individuos de tamanho tam
    populacao = []
    for i in range(N):
        populacao.append(geraTabuleiro(tam))
    return populacao

def crossOver(genoma1, genoma2):
    g1 = copy.deepcopy(genoma1)
    g2 = copy.deepcopy(genoma2)
    corte = random.randint(0,len(g1)-1)
    #dois filhos sao gerados
    return g1[:corte]+g2[corte:],g2[:corte]+g1[corte:]

def fitness(populacao):
    #retorna o indice de fitness da populacao inteira
    fits = []
    tam = len(populacao[0])
    maxAtaques = math.factorial(tam)/(math.factorial(tam-2)*2)
    for i in populacao:
        fits.append(1 - calcAtaques(i)/maxAtaques)
    return fits

def roletaViciada(populacao,fitness):
    #levantamento da qualidade dos individuos
    totalFitness = sum(fitness)
    #calculando as fracoes que cada individuo ocuparia na roleta
    fracoes = [f/totalFitness for f in fitness]
    #criando os intervalos da roleta
    intervalos = []
    acum = 0
    for i in fracoes:
        acum+=i
        intervalos.append(round(acum,2))
    #girando a roleta
    resultadoRoleta = random.randint(0,len(populacao))/len(populacao)
    #selecionando o individuo
    ind = 0
    while(resultadoRoleta>intervalos[ind]):
        ind+=1
    #retorna individuo selecionado
    return populacao[ind]

def selecaoNatural(populacao,numCasais):
    #selecao natural usando uma roleta viciada para escolha dos reprodutores
    selecao = []
    fits = fitness(populacao)
    for i in range(2*numCasais):
        selecao.append(roletaViciada(populacao,fits))
    return selecao

def testeMeta(pops):
    fits = fitness(pops)
    try:
        posMeta = fits.index(1)
        return "Solucao",pops[posMeta]
    except(ValueError):
        return -1

def genetico(maxIteracoes,tamanhoPopulacao,tamGenoma,taxaReproducao,deslocamentoGene):
    #|---------------------------------------------------------------------------|
    #| maxIteracoes     -> quantidade maxima de iteracoes                        |
    #| tamanhoPopulacao -> tamanho populacao inicial                             |
    #| tamGenoma        -> N x N                                                 |
    #| taxaReproducao   -> numero de casais a serem formados para reproducao     |
    #| deslocamentoGene -> passo no tabuleiro (regra de mutacao do genoma)       |
    #|---------------------------------------------------------------------------|

    #gera populacao inicial
    populacao = geraPopulacaoInicial(tamanhoPopulacao,tamGenoma)
    #iteracoes
    for i in range(maxIteracoes):
        ##print('Geração {}'.format(i))
        #primeiro teste de meta
        res = testeMeta(populacao)
        if(res!=-1):
            return res,i
        #etapa de evolucao por reproducao
        novaPopulacao = []
        reprodutores = selecaoNatural(populacao,taxaReproducao)
        machos = reprodutores[:taxaReproducao]
        femeas = reprodutores[taxaReproducao:]
        for k,w in zip(machos,femeas):
            f1,f2 = crossOver(k,w)
            novaPopulacao.append(f1)
            novaPopulacao.append(f2)
        #teste de meta na nova populacao
        res = testeMeta(novaPopulacao)
        if(res!=-1):
            return res,i
        #aceitar algumas mutacoes na nova populacao
        for z in novaPopulacao:
            if(random.randint(0,100)<10):
                z = mutacao(z,deslocamentoGene)
        populacao = novaPopulacao
    return ["Falha",populacao],-1

def main():
    #result = genetico(100, 10, 4, 5, 3) # 4x4
    result = genetico(10000, 20, 8, 10, 3) # 8x8

    if(result[0][0]=='Solucao'):
        print('Solucao (Geracao:{})'.format(result[1]))
        for i in result[0][1]:
            print(i)
        return True
    else:
        print('Falha')
        return False

if __name__=='__main__':
    main()


Solucao (Geracao:2202)
[0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0]
